In [1]:
import pandas as pd
import numpy as np
import itertools 
import matplotlib.pyplot as plt

In [ ]:
%run assignment_with_changes.ipynb

iteration: 1


<ipython-input-2-f2bce92bf0f6>:30: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  factoriz=pd.DataFrame(np.matmul(M, U), index=x.index, columns= x.columns)


iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
iteration: 35
iteration: 36
iteration: 37
iteration: 38
iteration: 39
iteration: 40
iteration: 41
iteration: 42
iteration: 43
iteration: 44
iteration: 45
iteration: 46
iteration: 47
iteration: 48
iteration: 49
iteration: 50
iteration: 51
iteration: 52
iteration: 53
iteration: 54
iteration: 55
iteration: 56
iteration: 57
iteration: 58
iteration: 59
iteration: 60
iteration: 61
iteration: 62
iteration: 63
iteration: 64
iteration: 65
iteration: 66
iteration: 67
iteration: 68
iteration: 69
iteration: 70
iteration: 71


In [ ]:
# Find Id of movie with ratings
rated_id = [id in np.array(ratings[['MovieID']]) for id in np.array(movies[['MovieID']])]
movies_rated = movies.loc[rated_id]

In [ ]:
dates = [s[-5:-1] for s in movies_rated.loc[:,'Title']]

In [ ]:
genres = [s.split('|')[0] for s in movies_rated.loc[:,'Genres']]

In [ ]:
np.unique(genres)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Start by visualizing users
pca = PCA(n_components=2)

PCs = pca.fit_transform(U.transpose())

In [ ]:
# Visualize users labelled by genders
plt.figure(figsize=(10,10)) 

for gender in ['M', 'F']:
    mask = np.array(users[['Gender']] == gender)
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=gender, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize users labelled by age
plt.figure(figsize=(10,10)) 
ages = np.unique(users['Age'])

for age in ages:
    mask = np.array(users[['Age']] == age)
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=age, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by genre
pca = PCA(n_components=2)

PCs = pca.fit_transform(M)

plt.figure(figsize=(10,10)) 

for genre in genres:
    mask = np.array(genres) == genre
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=genre, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by year of release
plt.figure(figsize=(10,10)) 

for date in dates:
    mask = np.array(dates) == date
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=date, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

## UMAP

In [ ]:
import umap

In [ ]:
# Visualize users labelled by genders
umap = umap.UMAP()
X_u = umap.fit_transform(U.transpose())

In [ ]:
plt.figure(figsize=(10,10)) 

for gender in ['M', 'F']:
    mask = np.array(users[['Gender']]) == gender
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(X_u[mask, 0], X_u[mask, 1], label=gender, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize users labelled by age
plt.figure(figsize=(10,10)) 
ages = np.unique(users['Age'])

for age in ages:
    mask = np.array(users[['Age']]) == age
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(X_u[mask, 0], X_u[mask, 1], label=age, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by genre
umap = umap.UMAP()
X_m = umap.fit_transform(M)

plt.figure(figsize=(10,10)) 

for genre in genres:
    mask = np.array(genres) == genre
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(X_m[mask, 0], X_m[mask, 1], label=genre, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by year of release
plt.figure(figsize=(10,10)) 

for date in dates:
    mask = np.array(dates) == date
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(X_m[mask, 0], X_m[mask, 1], label=date, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

## T-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Visualize users labelled by genders
t_sne = TSNE(n_components=2)
U_tsne = t_sne.fit_transform(U.transpose())

plt.figure(figsize=(10,10)) 

for gender in ['M', 'F']:
    mask = np.array(users[['Gender']]) == gender
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(U_tsne[mask, 0], U_tsne[mask, 1], label=gender, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize users labelled by age
plt.figure(figsize=(10,10)) 
ages = np.unique(users['Age'])

for age in ages:
    mask = np.array(users[['Age']]) == age
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(U_tsne[mask, 0], U_tsne[mask, 1], label=age, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by genre
t_sne = TSNE(n_components=2)
M_tsne = t_sne.fit_transform(M)

plt.figure(figsize=(10,10)) 

for genre in genres:
    mask = np.array(genres) == genre
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(M_tsne[mask, 0], M_tsne[mask, 1], label=genre, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

In [ ]:
# Visualize movies labelled by year of release
plt.figure(figsize=(10,10)) 

for date in dates:
    mask = np.array(dates) == date
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(M_tsne[mask, 0], M_tsne[mask, 1], label=date, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)